We want to be able to:

- connect Python with a MySQL database
- be able to manage a MYSQL database from Python (e.g. creating databases and tables, inserting data, deleting tables, ...)
- write Pandas DataFrames to a MySQL database
- read data from a MySQL database into a Pandas DataFrame
- use SQL queries to filter data in a MySQL database

In [1]:
# %pip install SQLAlchemy mysql-connector-python python-dotenv 

In [2]:
from sqlalchemy import create_engine, text   # Needed to connect to the database
import pandas as pd
import os                                    # Needed to access environment variables
from dotenv import load_dotenv               # Load passwords etc from .env file 

## 1.1 Connect to MySQL Database

- Instead of writing your database credentials into a Python script or Jupyter Notebook, you can store them in a separate .env file. 
- This is a text file that contains environment variables in the form of name=value pairs. 
- You can then load these variables into your Python script using the dotenv package.
- Such a .env file should never be shared with others or checked into version control.

In [3]:
# Load the .env file
load_dotenv('.env')

True

- The package SQLAlchemy allows you to connect to different SQL databases (e.g. MySQL, PostgreSQL, SQLite, ...).
- To connect to a MySQL database, you need to specify both the database dialect (mysql) and choose an appropriate driver (mysqlconnector, pymysql, ...). The driver needs to be installed separately. [MySQL Connector/Python](https://dev.mysql.com/doc/connector-python/en/connector-python-introduction.html) is the official MySQL driver for Python. 

In [4]:
# Connection to local MYSQL database
DIALECT = 'mysql'
DRIVER = 'mysqlconnector'                          # pip install mysql-connector-python
USER = os.getenv('MYSQL_USER')
PASSWORD = os.getenv('MYSQL_PASSWORD')
HOST = 'localhost'
PORT = '3306'

connection_string = f"{DIALECT}+{DRIVER}://{USER}:{PASSWORD}@{HOST}:{PORT}"
engine = create_engine(connection_string)

## 1.2 Create a new database

- We can use the `execute` method to run arbitrary SQL statements
- To make sure that the connection to the database is closed after the transaction, we can use the `with` statement. 

In [6]:
with engine.connect() as connection:
    connection.execute(text('CREATE DATABASE IF NOT EXISTS music'))

- Typically, we would not create a new database in this way, but use a database that already exists.
- In this case, we would directly specify an engine that points to the existing database.

In [7]:
connection_string = f"{DIALECT}+{DRIVER}://{USER}:{PASSWORD}@{HOST}:{PORT}/music"
engine = create_engine(connection_string)

## 1.3 Write Pandas DataFrame to database

- We use the `to_sql` method of Pandas DataFrames to write an entire DataFrame to a database table
- We can specify how to handle the situation that the table already exists: 'fail', 'replace', 'append'
- Also we need to consider whether the index should be written to the database (default)

In [8]:
df = pd.read_csv("data/tracks.csv")

In [9]:
df.to_sql(name='tracks', con=engine, if_exists='replace', index=False)

792

## 1.4 Read from database into Pandas DataFrame

- We can read entire database tables

In [9]:
tracks = pd.read_sql('tracks', con=engine)
tracks.head()

,id,name,album_name,artist_ids,danceability,energy,speechiness,acousticness,valence,tempo,duration_ms
0,05Mp2UJulSttxQ4E6hQPH3,Ohne mein Team,Palmen aus Plastik,"1aS5tqEs9ci5P9KD9tZWa6,0Dvx6p8JDyzeOPGmaCIH1L,...",0.766,0.800,0.0938,0.160,0.635,129.999,188504
1,4bHsxqR3GMrXTxEPLuK5ue,Don't Stop Believin',Escape,0rvjqX7ttXeg3mTy8Xscbt,0.500,0.748,0.0363,0.127,0.514,118.852,250987
2,3rdAz1fbUfZxYgaCviYhRo,Todo De Ti,VICE VERSA,1mcTU81TzQhprhouKaTkpq,0.780,0.719,0.0506,0.302,0.336,127.962,199604
3,254bXAqt3zP6P50BdQvEsq,Everywhere - 2017 Remaster,Tango In the Night (Deluxe Edition),08GQAI4eElDnROBrJRGE0X,0.730,0.487,0.0303,0.258,0.731,114.965,226653
4,2PGA1AsJal6cyMNmKyE56q,200 km/h,Platte,1qQLhymHXFPtP5U8KNKsm6,0.899,0.670,0.1630,0.269,0.413,148.065,163147


- ... and send arbitrary SQL queries to read data into a Pandas DataFrame

In [10]:
pd.read_sql('select name, album_name, danceability from tracks order by danceability desc limit 5', con=engine)

,name,album_name,danceability
0,Pure Cocaine,Street Gossip,0.964
1,Yes Indeed,Harder Than Ever,0.963
2,Low Down,My Turn (Deluxe),0.962
3,CAIRO,MAÑANA SERÁ BONITO,0.957
4,Starlight,Starlight,0.954


In [11]:
pd.read_sql('select count(*), avg(danceability) from tracks', con=engine)

,count,avg
0,792,0.695527


## 1.5 Delete database and clean up

- Now we want to undo the previous steps, and clean up
- Caution: this is irreversible!
- First, we drop the newly created table "tracks" within our "music" database

In [10]:
with engine.connect() as connection:
    connection.execute(text("Drop table tracks"))

- Second, we drop the entire database "music"

In [11]:
connection_string = f"{DIALECT}+{DRIVER}://{USER}:{PASSWORD}@{HOST}:{PORT}"
engine = create_engine(connection_string)
with engine.connect() as connection:
    connection.execute(text('DROP DATABASE music'))

- To make sure that all open connections to the database are closed, we should explicitly close them at the end of the script

In [12]:
engine.dispose()